In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score
import tensorflow as tf

In [52]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train_pca.csv')
df_testing = pd.read_csv('../data/archive/test_pca.csv')

In [53]:
#Just getting the X Variables
x_train = df_training.iloc[:, :-1]


x_test = df_testing.iloc[:, :-1]


In [54]:
train_dep = pd.read_csv('../data/archive/train.csv')
test_dep = pd.read_csv('../data/archive/test.csv')

In [55]:
train_label = train_dep.iloc[:,-1]

train_dep = train_dep.iloc[:,-1]
test_dep = test_dep.iloc[:,-1]

In [56]:
train_dep = pd.get_dummies(train_dep)
test_dep = pd.get_dummies(test_dep)

In [57]:
def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=15, input_dim = 40,  activation='relu'),
        tf.keras.layers.Dense(units=40,  activation='relu'),
        tf.keras.layers.Dense(units=6,  activation='softmax')])
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [58]:
ann = build_model()

In [59]:
ann.fit(x_train, train_dep, batch_size = 5, epochs = 100, verbose=0)

In [60]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)*1
test_dep = test_dep.to_numpy()

In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy_score(test_dep, y_pred)

0.8992195453003053

In [62]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

keras_clf = KerasClassifier(build_fn=build_model, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cross_val_accuracies = cross_val_score(estimator=keras_clf, scoring="accuracy", 
    X=x_train, y=train_label, cv=kfold)

In [63]:
cross_val_accuracies

array([0.96059783, 0.94972826, 0.96462585, 0.96734694, 0.97142857,
       0.97823129, 0.95238095, 0.97142857, 0.96190476, 0.95238095])